In [40]:
import pandas_datareader.data as web
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
asset_list = ['AAPL', 'NFLX','AMZN','GOOGL']

ind5 = web.get_data_yahoo(asset_list, start='2016-01-01', end='2019-01-01')['Adj Close']

ind5_ret = ind5.pct_change().dropna()
ind5_ret.head()


Symbols,AAPL,NFLX,AMZN,GOOGL
Date,,,,
2016-01-04,0.000855,-0.038643,-0.057554,-0.023869
2016-01-05,-0.025059,-0.020917,-0.005024,0.002752
2016-01-06,-0.019570,0.093071,-0.001799,-0.002889
2016-01-07,-0.042205,-0.026513,-0.039058,-0.024140
2016-01-08,0.005288,-0.027671,-0.001464,-0.013617


In [42]:
asset_num = len(asset_list)
asset_list, asset_num

(['AAPL', 'NFLX', 'AMZN', 'GOOGL'], 4)

In [43]:
mu = ind5_ret.mean() * ind5.shape[0] ## 1년 단위로 mu를 계산하는가?
sigma = ind5_ret.cov() * ind5.shape[0] 

display(mu)
display(sigma)

Symbols
AAPL     0.541938
NFLX     1.095881
AMZN     0.930050
GOOGL    0.367117
dtype: float64

Symbols,AAPL,NFLX,AMZN,GOOGL
Symbols,,,,
AAPL,0.167769,0.122269,0.111574,0.090208
NFLX,0.122269,0.490705,0.195084,0.142356
AMZN,0.111574,0.195084,0.261696,0.132201
GOOGL,0.090208,0.142356,0.132201,0.142960


#### Cvxpy

In [44]:
import cvxpy as cp

In [45]:
### GMV 포트폴리오 찾기 
gmv_ns2 = cp.Variable(asset_num)
objective = cp.Minimize(cp.quad_form(gmv_ns2, sigma))
constraints = [cp.sum(gmv_ns2) == 1, gmv_ns2 >= 0]
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS)
gmv_ns2 = gmv_ns2.value
gmv_ns2

array([4.04808802e-01, 2.49000184e-09, 1.37837926e-08, 5.95191182e-01])

In [46]:
# GMV return and risk (no short)
np.dot(mu.T, gmv_ns2), np.sqrt(np.dot(gmv_ns2.T, np.dot(sigma, gmv_ns2)))

(0.4378860811114345, 0.3487199944239129)

## MV  : 분산 최소화, 특정 수익률 이상 얻기 
###  Portfolios (no-shorting)

In [47]:
# GMV수익률보다 2퍼센트 더 높은 수익률을 원한다!
ret_gmv = np.dot(mu.T, gmv_ns2)
ret_target = np.round(ret_gmv,2) + 0.02 
ret_target

0.46

In [48]:
w = cp.Variable(asset_num)
objective = cp.Minimize(cp.quad_form(w, sigma)) # 목적함수 : 분산 최소화 
constraints = [cp.sum(w) == 1, # weight합은 1
               w >= 0,  # weight 모두 0보다 크다 (공매도 ㄴㄴ)
               w @ mu >= ret_target] ## @는 행렬 연산, mu가 ret_target보다 높게끔 제약조건 추가 
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS)
w = w.value
w

array([4.14062609e-01, 3.41850321e-09, 3.64096374e-02, 5.49527750e-01])

In [49]:
# 최종적으로 포트폴리오 기대 수익률, 표준편차 
np.dot(mu.T, w), np.sqrt(np.dot(w.T, np.dot(sigma, w)))

(0.4600000013387703, 0.3492676493228597)

## back test

In [50]:
asset_list = ['AAPL', 'NFLX','AMZN','GOOGL']

future = web.get_data_yahoo(asset_list, start='2019-01-01', end='2020-01-01')['Adj Close']

future_ret = future.pct_change().dropna()
future_ret.head()

Symbols,AAPL,NFLX,AMZN,GOOGL
Date,,,,
2019-01-02,0.001141,0.000000,0.024741,0.009302
2019-01-03,-0.099607,0.013226,-0.025242,-0.027696
2019-01-04,0.042689,0.097234,0.050064,0.051294
2019-01-07,-0.002226,0.059717,0.034353,-0.001994
2019-01-08,0.019063,0.015634,0.016612,0.008783


In [51]:
future_mu = future_ret.mean() * future.shape[0] 
future_sigma = ind5_ret.cov() * future.shape[0]


In [52]:
print('# 같은 비율로 투자했을 때 포트폴리오 평가 ')
# Portfolio mean, variance, and standard deviation
pfo_mu = np.dot(future_mu, w)
pfo_var = np.dot(np.dot(w.T, future_sigma), w)
pfo_var2 = np.dot(w.T, np.dot(future_sigma, w))
pfo_std = np.sqrt(pfo_var)


display(pfo_mu, pfo_var, pfo_var2, pfo_std)


print('# Monthly mean and volatility')
display(pfo_mu / 12, pfo_std / np.sqrt(12))

# 같은 비율로 투자했을 때 포트폴리오 평가 


0.4400812636350133

0.04087806144168156

0.04087806144168156

0.20218323729152612

# Monthly mean and volatility


0.03667343863625111

0.0583652732379463